# IMPORTING LIBRARIES


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# INPUT AND NORMALIZATION


In [ ]:
# Loading the data
file_path = ""
df = pd.read_excel(file_path)
confirmed_cases = df['Confirmed Cases'].values
df.rename(columns={'Confirmed Cases': 'E', 'Active Cases': 'I', 'Cured/Discharged': 'R', 'Death': 'D'}, inplace=True)

E = df['E'].values
I = df['I'].values
R = df['R'].values
D = df['D'].values
mean_E = np.mean(E)
std_E = np.std(E)
mean_I = np.mean(I)
std_I = np.std(I)
mean_R = np.mean(R)
std_R = np.std(R)
mean_D = np.mean(D)
std_D = np.std(D)

initial_S = 138.31e7 #Indian population at the time when covid - 19 hit
S = initial_S - confirmed_cases


mean_S = np.mean(S)
std_S = np.std(S)

# Calculating z-scores
def calculate_z_scores(data):
    mean = np.mean(data)
    std = np.std(data)
    z_scores = (data - mean) / std
    return z_scores

S_z_scores = calculate_z_scores(S)
E_z_scores = calculate_z_scores(E)
I_z_scores = calculate_z_scores(I)
R_z_scores = calculate_z_scores(R)
D_z_scores = calculate_z_scores(D)

# Converting z-scores to torch tensors
S_z = torch.tensor(S_z_scores, dtype=torch.float32).reshape(-1, 1)
E_z = torch.tensor(E_z_scores, dtype=torch.float32).reshape(-1, 1)
I_z = torch.tensor(I_z_scores, dtype=torch.float32).reshape(-1, 1)
R_z = torch.tensor(R_z_scores, dtype=torch.float32).reshape(-1, 1)
D_z = torch.tensor(D_z_scores, dtype=torch.float32).reshape(-1, 1)

#Debugging Steps(If needed)
print(S_z)
print(E_z)
print(I_z)
print(R_z)
print(D_z)

# NEURAL NETWORK MODEL


In [7]:
class ParameterModel(nn.Module):
    def __init__(self):
        super(ParameterModel, self).__init__()
        self.fc1 = nn.Linear(1, 35)
        self.fc2 = nn.Linear(35, 50)
        self.fc3 = nn.Linear(50, 30)
        self.fc4 = nn.Linear(30, 30)
        self.fc5 = nn.Linear(30, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        #print(f"After fc1: {x}")
        x = self.relu(self.fc2(x))
        #print(f"After fc2: {x}")
        x = self.relu(self.fc3(x))
        #print(f"After fc3: {x}")
        x = self.relu(self.fc4(x))
        #print(f"After fc4: {x}")
        x = self.sigmoid(self.fc5(x))
        #print(f"After fc5: {x}")
        return x


# RUNGE KUTTA METHOD FOR SOLVING ODEs


In [8]:
# Model using the fourth-order Runge-Kutta method
def seird_model_rk4(S, E, I, R, D, alpha, beta, gamma, mu, dt=1):
    def derivatives(S, E, I, R, D, alpha, beta, gamma, mu):
        dS_dt = -alpha * S * I
        dE_dt = alpha * S * I - gamma * E  - mu * E - beta * E
        dI_dt = beta * E
        dR_dt = gamma * E
        dD_dt = mu * E
        return dS_dt, dE_dt, dI_dt, dR_dt, dD_dt

    K1_S, K1_E, K1_I, K1_R, K1_D = derivatives(S, E, I, R, D, alpha, beta, gamma, mu)
    K2_S, K2_E, K2_I, K2_R, K2_D = derivatives(S + 0.5 * dt * K1_S, E + 0.5 * dt * K1_E, I + 0.5 * dt * K1_I, R + 0.5 * dt * K1_R, D + 0.5 * dt * K1_D, alpha, beta, gamma, mu)
    K3_S, K3_E, K3_I, K3_R, K3_D = derivatives(S + 0.5 * dt * K2_S, E + 0.5 * dt * K2_E, I + 0.5 * dt * K2_I, R + 0.5 * dt * K2_R, D + 0.5 * dt * K2_D, alpha, beta, gamma, mu)
    K4_S, K4_E, K4_I, K4_R, K4_D = derivatives(S + 0.5 * dt * K3_S, E + dt * K3_E, I + dt * K3_I, R + dt * K3_R, D + dt * K3_D, alpha, beta, gamma, mu)

    S_next = S + (dt / 6) * (K1_S + 2 * K2_S + 2 * K3_S + K4_S)
    E_next = E + (dt / 6) * (K1_E + 2 * K2_E + 2 * K3_E + K4_E)
    I_next = I + (dt / 6) * (K1_I + 2 * K2_I + 2 * K3_I + K4_I)
    R_next = R + (dt / 6) * (K1_R + 2 * K2_R + 2 * K3_R + K4_R)
    D_next = D + (dt / 6) * (K1_D + 2 * K2_D + 2 * K3_D + K4_D)

    return S_next, E_next, I_next, R_next, D_next

# LOSS FUNCTION CALCULATION


In [9]:
# Loss function
def loss_function(S_true, E_true, I_true, R_true, D_true, S_pred, E_pred, I_pred, R_pred, D_pred, omega_S, omega_E, omega_I, omega_R, omega_D, omega_params):
    loss_S = torch.mean(torch.abs(S_true - S_pred) ** 2) * omega_S
    loss_E = torch.mean(torch.abs(E_true - E_pred) ** 2) * omega_E
    loss_I = torch.mean(torch.abs(I_true - I_pred) ** 2) * omega_I
    loss_R = torch.mean(torch.abs(R_true - R_pred) ** 2) * omega_R
    loss_D = torch.mean(torch.abs(D_true - D_pred) ** 2) * omega_D
    regularization_loss = omega_params * (torch.sum(alpha_model.fc1.weight**2) + torch.sum(beta_model.fc1.weight**2) + torch.sum(gamma_model.fc1.weight**2) + torch.sum(mu_model.fc1.weight**2))
    total_loss = loss_S + loss_E + loss_I + loss_R + loss_D + regularization_loss
    return total_loss

# TRAINING THE MODEL

In [ ]:
# Training
def train_SEIRD_model(max_epoch, S_data, E_data, I_data, R_data, D_data, alpha_model, beta_model, gamma_model, mu_model, omega_S, omega_E, omega_I, omega_R, omega_D, omega_params):
    optimizer = optim.Adam(list(alpha_model.parameters()) + list(beta_model.parameters()) + list(gamma_model.parameters()) + list(mu_model.parameters()), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)

    total_loss_values = []
    log_loss_values = []

    for epoch in range(max_epoch):
        optimizer.zero_grad()

        alpha = alpha_model(S_data)
        beta = beta_model(E_data)
        gamma = gamma_model(I_data)
        mu = mu_model(R_data)

        S_pred, E_pred, I_pred, R_pred, D_pred = seird_model_rk4(S_data, E_data, I_data, R_data, D_data, alpha, beta, gamma, mu)
        loss = loss_function(S_data, E_data, I_data, R_data, D_data, S_pred, E_pred, I_pred, R_pred, D_pred, omega_S, omega_E, omega_I, omega_R, omega_D, omega_params)

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss_values.append(loss.item())
        log_loss_values.append(torch.log(loss).item())

        if epoch % 1 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    return total_loss_values, log_loss_values

# Hyperparameters
max_epoch = 100
# Change according to compartment weightage
omega_S = 1.0
omega_E = 1.0
omega_I = 1.0
omega_R = 1.0
omega_D = 1.0
omega_params = 0.0005

# Initialize models
alpha_model = ParameterModel()
beta_model = ParameterModel()
gamma_model = ParameterModel()
mu_model = ParameterModel()

total_loss_values, log_loss_values = train_SEIRD_model(max_epoch, S_z, E_z, I_z, R_z, D_z, alpha_model, beta_model, gamma_model, mu_model, omega_S, omega_E, omega_I, omega_R, omega_D, omega_params)

# Function to predict SEIRD values
def predict_seird(S, E, I, R, D, alpha_model, beta_model, gamma_model, mu_model):
    alpha = alpha_model(S)
    beta = beta_model(E)
    gamma = gamma_model(I)
    mu = mu_model(R)
    S_pred, E_pred, I_pred, R_pred, D_pred = seird_model_rk4(S, E, I, R, D, alpha, beta, gamma, mu)
    return S_pred, E_pred, I_pred, R_pred, D_pred

S_pred, E_pred, I_pred, R_pred, D_pred = predict_seird(S_z, E_z, I_z, R_z, D_z, alpha_model, beta_model, gamma_model, mu_model)

# Converting predicted values from tensors to numpy arrays
S_pred = S_pred.detach().numpy()
E_pred = E_pred.detach().numpy()
I_pred = I_pred.detach().numpy()
R_pred = R_pred.detach().numpy()
D_pred = D_pred.detach().numpy()

In [40]:
def denormalize_data(normalized_data, mean, std):
    denormalized_data = (normalized_data * std) + mean
    return denormalized_data
# Denormalizing predicted values
S_pred_denormalized = denormalize_data(S_pred, mean_S, std_S)
E_pred_denormalized = denormalize_data(E_pred, mean_E, std_E)
I_pred_denormalized = denormalize_data(I_pred, mean_I, std_I)
R_pred_denormalized = denormalize_data(R_pred, mean_R, std_R)
D_pred_denormalized = denormalize_data(D_pred, mean_D, std_D)


In [ ]:
# Plotting the actual vs predicted values
plt.figure(figsize=(14, 8))

plt.subplot(3, 2, 1)
plt.plot(S, label='Original S', color='blue')
plt.plot(S_pred_denormalized, label='S-Predicted', color='orange')
plt.title('Susceptible Population(S)')
plt.xlabel('Time')
plt.ylabel('People')
plt.legend()

plt.subplot(3, 2, 2)
plt.plot(E, label='Original E', color='blue')
plt.plot(E_pred_denormalized, label='E-Predicted', color='orange')
plt.title('Exposed (E)')
plt.xlabel('Time')
plt.ylabel('People')
plt.legend()

plt.subplot(3, 2, 3)
plt.plot(I, label='Original I', color='blue')
plt.plot(I_pred_denormalized, label='I-Predicted', color='orange')
plt.title('Infected (I)')
plt.xlabel('Time')
plt.ylabel('People')
plt.legend()

plt.subplot(3, 2, 4)
plt.plot(R, label='Original R', color='blue')
plt.plot(R_pred_denormalized, label='R-Predicted', color='orange')
plt.title('Recovered (R)')
plt.xlabel('Time')
plt.ylabel('People')
plt.legend()

plt.subplot(3, 2, 5)
plt.plot(D, label='Original D', color='blue')
plt.plot(D_pred_denormalized, label='D-Predicted', color='orange')
plt.title('Deaths (D)')
plt.xlabel('Time')
plt.ylabel('People')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt

alpha_model.eval()
beta_model.eval()
gamma_model.eval()
mu_model.eval()

S_pred_tensor = torch.tensor(S_z, dtype=torch.float32)
E_pred_tensor = torch.tensor(E_z, dtype=torch.float32)
I_pred_tensor = torch.tensor(I_z, dtype=torch.float32)
R_pred_tensor = torch.tensor(R_z, dtype=torch.float32)

with torch.no_grad():  # Disable gradient calculation
    alpha_values = alpha_model(S_pred_tensor).numpy()
    beta_values = beta_model(E_pred_tensor).numpy()
    gamma_values = gamma_model(I_pred_tensor).numpy()
    mu_values = mu_model(R_pred_tensor).numpy()

#Debugging Steps(If needed)
print("Predicted Beta values (first array):\n", beta_values)
print("Predicted Gamma values (first array):\n", gamma_values)
print("Predicted Mu values (first array):\n", mu_values)

# Plotting the predicted values
time_steps = range(len(E_z))

plt.figure(figsize=(14, 8))

# Plotting Beta values
plt.subplot(4, 1, 1)
plt.plot(time_steps, beta_values, label='Beta', color='blue')
plt.xlabel('Time Steps')
plt.ylabel('Beta')
plt.title('Predicted Beta Values')
plt.legend()

# Plotting Gamma values
plt.subplot(4, 1, 2)
plt.plot(time_steps, gamma_values, label='Gamma', color='green')
plt.xlabel('Time Steps')
plt.ylabel('Gamma')
plt.title('Predicted Gamma Values')
plt.legend()

# Plotting Mu values
plt.subplot(4, 1, 3)
plt.plot(time_steps, mu_values, label='Mu', color='red')
plt.xlabel('Time Steps')
plt.ylabel('Mu')
plt.title('Predicted Mu Values')
plt.legend()

# Plotting Alpha values
plt.subplot(4, 1, 4)
plt.plot(time_steps, alpha_values, label='Alpha', color='orange')
plt.xlabel('Time Steps')
plt.ylabel('Alpha')
plt.title('Predicted Alpha Values')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plotting the SEIRD graph using predicted values
plt.figure(figsize=(10, 6))

# Plotting predicted values for each compartment separately
plt.plot(S_pred_denormalized, label='S')
plt.plot(E_pred_denormalized, label='E')
plt.plot(I_pred_denormalized, label='I')
plt.plot(R_pred_denormalized, label='R')
plt.plot(D_pred_denormalized, label='D')

plt.title('Predicted SEIRD Values')
plt.xlabel('Time')
plt.ylabel('Number of Cases')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plotting the EIRD graph using predicted values
plt.figure(figsize=(10, 6))

# Plotting predicted values for each compartment separately
plt.plot(E_pred_denormalized, label='E')
plt.plot(I_pred_denormalized, label='I')
plt.plot(R_pred_denormalized, label='R')
plt.plot(D_pred_denormalized, label='D')

plt.title('Predicted EIRD Values')
plt.xlabel('Time')
plt.ylabel('Number of Cases')
plt.legend()
plt.grid(True)
plt.show()
